In [7]:
import pandas as pd
import string
import os
import numpy as np
from sqlalchemy import create_engine, text

from backend.utils import DbEngine, Queries
from backend.etl import ETL
from backend.etl_sdhi import ETL_SDHI
from encoding_module.encoder import Encoding

ValueError: attempted relative import beyond top-level package

### Connect to DB

In [10]:
db_engine = DbEngine(user = 'team77', 
                     password = 'mintic2020.',
                     ip = 'postgres', 
                     port = '5432', 
                     db = 'minjusticia')
engine = db_engine.connect()
queries = Queries(engine)

In [12]:
db_engine = DbEngine(user = 'postgres', 
                     password = 'YyjnDpcVRtpHDOHHzr58',
                     ip = 'database-1.cjppulxuzu8c.us-east-2.rds.amazonaws.com', 
                     port = '5432', 
                     db = 'minjusticia')
engine = db_engine.connect()
queries = Queries(engine)

### Loading original data

In [3]:
data = pd.read_excel('../data/ReincidenciaPospenadosNal201011Junio2020Rev.xlsx', skiprows = 6)

In [4]:
data.count()

INTERNOEN                 142899
DELITO                    142899
TITULO_DELITO             142899
SUBTITULO_DELITO          142899
TENTATIVA                 142899
AGRAVADO                  142899
CALIFICADO                142899
FECHA_INGRESO             142899
FECHA_SALIDA               96345
FECHA_CAPTURA             142899
SITUACION_JURIDICA        142899
ANO_NACIMIENTO            142899
EDAD                      142899
GENERO                    142899
ESTADO_CIVIL               62810
PAIS_INTERNO              139737
DEPARTAMENTO              142892
CIUDAD                    142899
REINCIDENTE               142899
ESTADO_INGRESO            142869
ACTIVIDADES_TRABAJO       142899
ACTIVIDADES_ESTUDIO       142899
ACTIVIDADES_ENSEÑANZA     142899
NIVEL_EDUCATIVO           142899
HIJOS_MENORES             142899
CONDIC_EXPECIONAL          13314
CODIGO_ESTABLECIMIENTO    142899
ESTABLECIMIENTO           142899
DEPTO_ESTABLECIMIENTO     140108
MPIO_ESTABLECIMIENTO      140108
REGIONAL  

In [5]:
data2 = data[['INTERNOEN', 'DELITO', 'TITULO_DELITO', 'SUBTITULO_DELITO', 'TENTATIVA',
       'AGRAVADO', 'CALIFICADO', 'FECHA_INGRESO', 'FECHA_SALIDA',
       'FECHA_CAPTURA', 'SITUACION_JURIDICA', 'ANO_NACIMIENTO', 'EDAD',
       'GENERO', 'ESTADO_CIVIL', 'PAIS_INTERNO', 'DEPARTAMENTO', 'CIUDAD',
       'REINCIDENTE', 'ESTADO_INGRESO', 'ACTIVIDADES_TRABAJO',
       'ACTIVIDADES_ESTUDIO', 'ACTIVIDADES_ENSEÑANZA', 'NIVEL_EDUCATIVO',
       'HIJOS_MENORES', 'CONDIC_EXPECIONAL', 'CODIGO_ESTABLECIMIENTO',
       'ESTABLECIMIENTO', 'DEPTO_ESTABLECIMIENTO', 'MPIO_ESTABLECIMIENTO',
       'REGIONAL', 'ESTADO']]
data2['AGRAVADO'].unique()

array(['N', 'S'], dtype=object)

### Execute ETL

In [6]:
etl = ETL(queries)
etl.inmate_static_info(data2)
etl.inmate_variable_info(data2)

KeyboardInterrupt: 

### Execute ETL SHDI index

In [43]:
data_shdi = pd.read_excel('../data/SHDI Colombia 4.0.xlsx')

In [44]:
data_shdi.columns

Index(['iso_code', 'country', 'year', 'GDLCODE', 'level', 'region',
       'continent', 'shdi', 'healthindex', 'incindex', 'edindex', 'lifexp',
       'gnic', 'esch', 'msch', 'pop'],
      dtype='object')

In [45]:
etl_sdhi = ETL_SDHI(queries)
etl_sdhi.inmate_static_info(data_shdi)

### Traslate reofenders predictive model

In [13]:
# Import data
df = pd.read_csv('../app-figures/reoffender_models.csv')
df.to_sql('reoffender_models', con=queries.engine)

KeyboardInterrupt: 

### Encode

In [18]:
encoding = Encoding(queries)
data_2_encode = encoding.get_data('encoding')
data_2_encode.columns

Index(['id_persona', 'internoen', 'genero', 'nacionalidad', 'reincidente',
       'anio_nacimiento', 'estado_civil', 'nivel_educativo',
       'reconocimiento_etnico', 'diversidad_sexual', 'extranjero',
       'condicion_exepcional', 'persona_id_persona', 'delito_id_delito',
       'estado_ingreso', 'id_registro', 'fecha_captura', 'fecha_ingreso',
       'establecimiento', 'tentativa', 'agravado', 'calificado',
       'fecha_salida', 'edad', 'municipio_id_municipio', 'actividades_estudio',
       'actividades_trabajo', 'actividades_enseñanza', 'hijos_menores',
       'condicion_excepcional', 'estado_id_estado', 'situacion_juridica',
       'madre_gestante', 'madre_lactante', 'discapacidad', 'adulto_mayor',
       'severity', 'shdi', 'healthindex', 'incindex', 'edindex', 'lifexp',
       'gnic', 'esch', 'msch', 'pop', 'id_delito', 'id_subtitulo_delito',
       'nombre', 'name_eng', 'severity'],
      dtype='object')

In [ ]:
data_encoded = encoding.one_hot(data_2_encode)
data_encoded

In [17]:
data_encoded.columns

Index(['internoen', 'genero', 'nacionalidad', 'estado_civil',
       'nivel_educativo', 'reconocimiento_etnico', 'diversidad_sexual',
       'extranjero', 'estado_ingreso', 'fecha_ingreso', 'establecimiento',
       'fecha_salida', 'edad', 'actividades_estudio', 'actividades_trabajo',
       'actividades_enseñanza', 'hijos_menores', 'condicion_excepcional',
       'estado_id_estado', 'madre_gestante', 'madre_lactante', 'discapacidad',
       'adulto_mayor', 'severity', 'severity', 'shdi', 'healthindex',
       'incindex', 'edindex', 'lifexp', 'gnic', 'esch', 'msch', 'pop',
       'severity', 'severity', 'delito_CONCIERTO PARA DELINQUIR',
       'delito_EXTORSION',
       'delito_FABRICACION  TRAFICO Y PORTE DE ARMAS Y MUNICIONES DE USO PRIVATIVO DE LAS FUERZAS ARMADAS',
       'delito_FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO O MUNICIONES',
       'delito_FABRICACIÓN, TRÁFICO, PORTE O TENENCIA DE ARMAS DE FUEGO, ACCESORIOS, PARTES O MUNICIONES',
       'delito_HOMICIDIO', 'delito_H